<a href="https://colab.research.google.com/github/defneyucesir/gametheory/blob/main/Tesseract_Graph_for_4_Player_Stag_Hunt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import plotly.graph_objects as go
from itertools import product

#Defining the verticies
vertices = np.array(list(product([-1, 1], repeat=4)))

#Defining the edges
edges = [(i, j) for i, v1 in enumerate(vertices) for j, v2 in enumerate(vertices) if np.sum(np.abs(v1 - v2)) == 2]

#Inputting pure and mixed strategy nash equilibria
pure = [np.array([-1, -1, -1, -1]), np.array([1, 1, 1, 1])]
msne = np.array([0.9, 0.9, 0.9, 0.9])

#Assigning indices to Nash equilibria before it rotates
pure_indices = [i for i, v in enumerate(vertices) if any(np.all(v == ne) for ne in pure)]

# Define best response edges based on the game logic
best_response_edges = []
for i, v1 in enumerate(vertices):
    for j, v2 in enumerate(vertices):
        if np.sum(np.abs(v1 - v2)) == 2 and np.any(np.array(v1) != np.array(v2)):
            best_response_edges.append((i, j))

#Rotating matrix in the z, w plane
def rotation_matrix_4D(theta):
    return np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, np.cos(theta), np.sin(theta)],
        [0, 0, -np.sin(theta), np.cos(theta)]
    ])

#Projecting perspective from 4d to 3d
def project(v):
    w_factor = 2 + v[3]  # Shifts w for perspective effect
    return np.array([v[0] / w_factor, v[1] / w_factor, v[2] / w_factor])

#Creating frames in animation so pure and mixed strategy equllibria appear at the correct time
frames = []
theta = np.linspace(0, 2 * np.pi, 50)

for theta in theta:
    rotated_vertices = np.dot(vertices, rotation_matrix_4D(theta).T)
    rotated_msne = np.dot(msne, rotation_matrix_4D(theta).T)
    projected_vertices = np.array([project(v) for v in rotated_vertices])
    projected_msne = project(rotated_msne)

    x, y, z = projected_vertices.T
    x_msne, y_msne, z_msne = projected_msne

    w_values = rotated_vertices[:, 3]

    #Mixed strategy nash equilbiria only appears when its w component is roughly 0
    msne_visible = np.abs(rotated_msne[3]) < 0.2

    #Assign colours, red for pure nash equilibria when w is near zero and blue for MSNE
    colors = [
        'red' if (i in pure_indices and np.abs(w_values[i]) < 0.2) else  #pure nash equilbira also only appear when w is roughly 0
        'pink'
        for i, v in enumerate(rotated_vertices)
    ]

    #Creating frame with updated vertices and MSNE
    frame = go.Frame(
        data=[
            go.Scatter3d(
                x=x, y=y, z=z,
                mode='markers',
                marker=dict(size=6, color=colors),
                hoverinfo='text'
            ),
            #Mixed strategy nash equilbria shows up as blue when aligned
            go.Scatter3d(
                x=[x_msne] if msne_visible else [],
                y=[y_msne] if msne_visible else [],
                z=[z_msne] if msne_visible else [],
                mode='markers',
                marker=dict(size=8, color='blue'),
                hoverinfo='text'
            ),
            #Adding edges of the tesseract
            *[
                go.Scatter3d(
                    x=[x[i], x[j]], y=[y[i], y[j]], z=[z[i], z[j]],
                    mode='lines',
                    line=dict(color='black', width=2),
                    showlegend=False
                )
                for i, j in edges
            ],

            *[
                go.Scatter3d(
                    x=[x[i], x[j]], y=[y[i], y[j]], z=[z[i], z[j]],
                    mode='lines',
                    line=dict(color='cyan', width=3, dash="dot"),
                    showlegend=False
                )
                for i, j in best_response_edges
            ]
        ]
    )
    frames.append(frame)

#projecting the initial version of the tesseract
initial_rotated_vertices = np.dot(vertices, rotation_matrix_4D(0).T)
initial_projected_vertices = np.array([project(v) for v in initial_rotated_vertices])
x_init, y_init, z_init = initial_projected_vertices.T
rotated_msne_initial = np.dot(msne, rotation_matrix_4D(0).T)
projected_msne_initial = project(rotated_msne_initial)


colors_init = [
    'red' if (i in pure_indices and np.abs(initial_rotated_vertices[i][3]) < 0.2) else
    'pink'
    for i in range(len(vertices))
]

#Animating the figure
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x_init, y=y_init, z=z_init,
            mode='markers',
            marker=dict(size=6, color=colors_init),
            hoverinfo='text'
        ),
        #MSNE appearing at start if the alignment is appropriate
        go.Scatter3d(
            x=[projected_msne_initial[0]] if np.abs(rotated_msne_initial[3]) < 0.2 else [],
            y=[projected_msne_initial[1]] if np.abs(rotated_msne_initial[3]) < 0.2 else [],
            z=[projected_msne_initial[2]] if np.abs(rotated_msne_initial[3]) < 0.2 else [],
            mode='markers',
            marker=dict(size=8, color='blue'),
            hoverinfo='text'
        ),
        #Setting initial tesseract edges
        *[
            go.Scatter3d(
                x=[x_init[i], x_init[j]], y=[y_init[i], y_init[j]], z=[z_init[i], z_init[j]],
                mode='lines',
                line=dict(color='black', width=2),
                showlegend=False
            )
            for i, j in edges
        ]
    ],
    layout=go.Layout(
        title="Visualising the Pure and Mixed Strategy Nash Equilibria in a Four-Player Stag Hunt ",
        scene=dict(xaxis_title='UK', yaxis_title='US', zaxis_title='China'),
        updatemenus=[{
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 50, "redraw": True}, "fromcurrent": True}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }]
    ),
    frames=frames
)

#Showing the graph
fig.show()
